## The Observer Pattern
The observer pattern is essentially a **Publisher** + **Subscriber** system. Publishers are also known as **Subject** and Subscribers are known as **Observers**.  

<img src="images/71tLE8e.jpg" width="700px" height="auto">

The Observer Pattern defines a one-to-many dependency between objects so that when one object changes state, all of its dependents are notified and updated automatically.  

<img src="images/3PtE8l0.jpg" width="500px" height="auto">

### Class Diagram
<img src="images/z2f4DhT.jpg" width="700px" height="auto">

The Subject and Observer classes are loosely coupled, they interact but have very little knowledge of each other.

## Implementation

In [2]:
public interface Observer {
    // The argument of the Observer depend upon what
    // data is emitted by the Subject
    // This however may not be the best approach, what if 
    // we now want to get another data? Update method
    // may need to be updated
    public void update(float p, float t, float v);
}

In [3]:
public interface Subject{
    public void registerObserver(Observer o);
    public void removeObserver(Observer o);
    public void notifyObservers();
}

In [4]:
public class Sensor implements Subject{
    private ArrayList<Observer> observers;
    private float pressure, temperature, volume;
    
    public Sensor(){
        observers = new ArrayList<>();
    }
    
    @Override
    public void registerObserver(Observer o){
        this.observers.add(o);
    }
    
    @Override
    public void removeObserver(Observer o){
        this.observers.remove(o);
    }
    
    @Override
    public void notifyObservers(){
        for(Observer observer: this.observers){
            observer.update(pressure, temperature, volume);
        }
    }
    
    public void measurementUpdated(){
        notifyObservers();
    }
    
    public void setMeasurement(float pressure, float temperature, float volume){
        this.pressure = pressure;
        this.temperature = temperature;
        this.volume = volume;
        
        measurementUpdated();
    }
}

In [7]:
public class DisplayDevice implements Observer{
    private float pressure, temperature, volume;
    // Hold reference to the subject in case you want to 
    // unsubscribe in the future, also helpful in pull based approach
    private Subject subject;
    private float pressure, temperature, volume;
    
    public DisplayDevice(Subject subject){
        this.subject = subject;
        subject.registerObserver(this);
    }
    
    @Override
    public void update(float p, float t, float v){
        this.pressure = p;
        this.temperature = t;
        this.volume = v;
        
        display();
    }
    
    public void display(){
        System.out.println("Pressure: " + pressure + " Temperature: " + temperature + " Volume: " + volume);
    }
}

The above implementation is not thread safe, one way to make it thread safe is to use a thread safe collection:

In [ ]:
private final Set<Observer> observers = Collections.newSetFromMap(new ConcurrentHashMap<Observer, Boolean>(0));

Another way is to use locks:

In [ ]:
public class Sensor implements Subject{
    private final Object MONITOR = new Object();
    private ArrayList<Observer> observers;
    private float pressure, temperature, volume;
    
    @Override
    public void registerObserver(Observer o){
        if(o == null) return;
        
        // Lazy init observers
        synchronized(MONITOR){
            if(observers == null) observers = new ArrayList<>();
            observers.add(o);
        }
    }
    
    @Override
    public void removeObserver(Observer o){
        if(o == null) return;
        
        synchronized(MONITOR){
            observers.remove(o);
        }
    }
    
    @Override
    public void notifyObservers(){
        synchronized(MONITOR){
            if(observers != null) {
                for(Observer observer: observers){
                    observer.update(pressure, temperature, volume);
                }
            }
        }
    }
    
    // ...
}

There is a problem with the above approach - while observers are being notified, all other operations add and remove observers are blocked to other threads, therefore, we modify the notify method as:

In [ ]:
@Override
public void notifyObservers(){
    ArrayList<Observer> copy;
    
    synchronized(MONITOR){
        if(observers == null) return;
        copy = new ArrayList<>(observers);
    }
    
    for(Observer observer: copy){
        observer.update(pressure, temperature, volume);
    }
}

There is still a problem that if a thread A is in the loop notifying observers and other thread removes an observer, the removed observer would still get notified.

Also, the better way to notify all observers is to use dedicated threads. Otherwise a long running observer update method may block other observers.

## Inbuilt Support in Java
Java has inbuilt support for Observer Pattern. The replacements for the above interfaces are:
- `java.util.Observable` class for Subject
- `java.util.Observer` interface for Observer

The `Observable` class has the following methods (not all listed):
- `addObserver(Observer o)`
- `deleteObserver(Observer o)`
- `notifyObservers()`
- `setChanged()`

**Pull vs Push Pattern:** The above code example illustrated *Push* pattern where Subject pushed updates to Observers. We can also have a *pull* pattern where the Observer query Subject if there is any new update. The Java implementation supports both patterns.

The observer interface has the following method:
- `update(Observable o, Object arg)`

In [8]:
public class Sensor extends Observable{
    // Since Observable is a class, addObserver, deleteObserver, etc
    // have already been implemented.

    // For an Observable to send notifications:
    // a) Call setChanged() to signal that new data has come
    // b) Call any of the notifyObservers() or notifyObservers(Object arg)
    //    first one calls update(this, null) other calls update(this, arg)

    private float pressure, temperature, volume;

    public void measurementUpdated(){
        setChanged();
        notifyObservers();
    }
    
    public void setMeasurement(float pressure, float temperature, float volume){
        this.pressure = pressure;
        this.temperature = temperature;
        this.volume = volume;
        
        measurementUpdated();
    }
}

In [ ]:
public class DisplayDevice implements Observer{
    private float pressure, temperature, volume;
    // Hold reference to the subject in case you want to 
    // unsubscribe in the future
    private Observable observable;
    private float pressure, temperature, volume;
    
    public DisplayDevice(Observable observable){
        this.observable = observable;
        observable.addObserver(this);
    }
    
    public void update(Observable o, Object arg){
        if (o instanceof Sensor){
            Sensor s = (Sensor) o;
            this.pressure = o.getPressure();
            this.temperature = o.getTemperature();
            this.volume = o.getVolume();

            display();
        }
    }
    
    public void display(){
        System.out.println("Pressure: " + pressure + " Temperature: " + temperature + " Volume: " + volume);
    }
}

`Observerable` and `Observer` have been deprecated since Java 9. Reasons:
- `Observable` doesn't implement `Serializable` and all its members are private. You can't serialize `Observable` neither its subclass.
- No thread safety
- They support only the notion that something has changed, but they don't convey any information about what has changed.